# FIND PNL

#### import library

In [2]:
import pandas as pd
import talib as ta
import numpy as np

#### import data for testing (csv) 

Trading view data (06/12/22 - 01/02/24) : 'EURUSD' timeframe 1 day 

In [3]:
ohlc = pd.read_csv(r'OANDA_EURUSD, 1D.csv')[['time','open','high','low','close','RSI_7']]

In [4]:
ohlc

,time,open,high,low,close,RSI_7
0,06-12-22,1.04671,1.05498,1.04434,1.05064,65.304797
1,07-12-22,1.05064,1.05650,1.04896,1.05552,70.251218
2,08-12-22,1.05552,1.05883,1.05036,1.05336,65.433901
3,11-12-22,1.05336,1.05803,1.05060,1.05378,65.963370
4,12-12-22,1.05378,1.06736,1.05281,1.06317,75.680013
...,...,...,...,...,...,...
295,28-01-24,1.08546,1.08546,1.07960,1.08333,32.202677
296,29-01-24,1.08333,1.08574,1.08120,1.08456,37.696732
297,30-01-24,1.08456,1.08876,1.07950,1.08176,31.020531
298,31-01-24,1.08176,1.08749,1.07800,1.08720,50.779340


#### use strategy with the data

In [5]:
def rsi_strategy (df_rsi, rsi_period=7, ovb=70, ovs=30, mid=50):
    
    df_rsi['rsi'] = ta.RSI(df_rsi['close'], timeperiod=rsi_period)
    df_rsi['rsi_lag'] = df_rsi['rsi'].shift(1)    
    
    current_rsi = df_rsi['rsi']
    previous_rsi = df_rsi['rsi_lag']
    
    crossoverbought = np.where((current_rsi >= ovb) & (previous_rsi < ovb), 'yes', 'no')
    crossoversold = np.where((current_rsi <= ovs) & (previous_rsi > ovs), 'yes', 'no')  

    df_rsi['crossovb'] = crossoverbought
    df_rsi['crossovs'] = crossoversold
    
    df_rsi.loc[df_rsi['crossovb'] == 'yes', 'action'] = 'buy'
    df_rsi.loc[df_rsi['crossovs'] == 'yes', 'action'] = 'sell'
    df_rsi.loc[df_rsi['action'] == 'nan', 'action'] = float('nan')
    
    exit_long = np.where((current_rsi <= mid) & (previous_rsi > mid), 'yes', 'no')
    exit_short = np.where((current_rsi >= mid) & (previous_rsi < mid), 'yes', 'no')

    df_rsi['exit_long'] = exit_long
    df_rsi['exit_short'] = exit_short
    
    df_rsi.loc[df_rsi['exit_long'] == 'yes', 'exit'] = 'exit long'
    df_rsi.loc[df_rsi['exit_short'] == 'yes', 'exit'] = 'exit short'
    df_rsi.loc[df_rsi['exit'] == 'nan', 'exit'] = float('nan')
    
    
    
    return df_rsi

In [6]:
ohlc_with_strategy = rsi_strategy(ohlc)
ohlc_with_strategy

,time,open,high,low,close,RSI_7,rsi,rsi_lag,crossovb,crossovs,action,exit_long,exit_short,exit
0,06-12-22,1.04671,1.05498,1.04434,1.05064,65.304797,NaN,NaN,no,no,NaN,no,no,NaN
1,07-12-22,1.05064,1.05650,1.04896,1.05552,70.251218,NaN,NaN,no,no,NaN,no,no,NaN
2,08-12-22,1.05552,1.05883,1.05036,1.05336,65.433901,NaN,NaN,no,no,NaN,no,no,NaN
3,11-12-22,1.05336,1.05803,1.05060,1.05378,65.963370,NaN,NaN,no,no,NaN,no,no,NaN
4,12-12-22,1.05378,1.06736,1.05281,1.06317,75.680013,NaN,NaN,no,no,NaN,no,no,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,28-01-24,1.08546,1.08546,1.07960,1.08333,32.202677,32.202677,37.052519,no,no,NaN,no,no,NaN
296,29-01-24,1.08333,1.08574,1.08120,1.08456,37.696732,37.696732,32.202677,no,no,NaN,no,no,NaN
297,30-01-24,1.08456,1.08876,1.07950,1.08176,31.020531,31.020531,37.696732,no,no,NaN,no,no,NaN
298,31-01-24,1.08176,1.08749,1.07800,1.08720,50.779340,50.779340,31.020531,no,no,NaN,no,yes,exit short


#### filter only row with action (entry and exit)

In [7]:
strategy_result = ohlc_with_strategy[(ohlc_with_strategy.action.notnull()) | (ohlc_with_strategy.exit_long=='yes') | (ohlc_with_strategy.exit_short=='yes')]

action_df = pd.DataFrame(strategy_result)[['time', 'open', 'high', 'low', 'close', 'rsi', 'action', 'exit']]

action_df.head(10)

,time,open,high,low,close,rsi,action,exit
16,29-12-22,1.06610,1.07133,1.06386,1.07039,74.234461,buy,NaN
18,02-01-23,1.06638,1.06835,1.05195,1.05490,39.271125,NaN,exit long
21,05-01-23,1.05215,1.06482,1.04833,1.06444,55.729759,NaN,exit short
25,11-01-23,1.07579,1.08670,1.07305,1.08514,74.369258,buy,NaN
32,22-01-23,1.08555,1.09270,1.08462,1.08718,72.063744,buy,NaN
39,31-01-23,1.08632,1.10013,1.08522,1.09896,77.472682,buy,NaN
41,02-02-23,1.09098,1.09404,1.07928,1.07940,40.539617,NaN,exit long
54,21-02-23,1.06464,1.06642,1.05992,1.06044,26.013443,sell,NaN
59,28-02-23,1.05753,1.06915,1.05653,1.06696,53.712348,NaN,exit short
60,01-03-23,1.06696,1.06730,1.05766,1.05969,42.345820,NaN,exit long


#### add position size, previous position, and sum to calculate for entry and exit signals matching

In [8]:
def get_position(action, exit, position = 0):
    if action == 'buy' or action == 'sell':
        position = position + 1
    elif exit == 'exit long' or exit == 'exit short':
        position = position - 1
    else:
        position = 0
    return position

action_df['position'] = list(map(get_position, action_df.action.values, action_df.exit.values))
action_df['position_lag'] = action_df['position'].shift(1).fillna(0)
action_df['position_sum'] = action_df['position'] + action_df['position_lag']

action_df

,time,open,high,low,close,rsi,action,exit,position,position_lag,position_sum
16,29-12-22,1.06610,1.07133,1.06386,1.07039,74.234461,buy,NaN,1,0.0,1.0
18,02-01-23,1.06638,1.06835,1.05195,1.05490,39.271125,NaN,exit long,-1,1.0,0.0
21,05-01-23,1.05215,1.06482,1.04833,1.06444,55.729759,NaN,exit short,-1,-1.0,-2.0
25,11-01-23,1.07579,1.08670,1.07305,1.08514,74.369258,buy,NaN,1,-1.0,0.0
32,22-01-23,1.08555,1.09270,1.08462,1.08718,72.063744,buy,NaN,1,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
276,01-01-24,1.10374,1.10453,1.09383,1.09406,43.451205,NaN,exit long,-1,1.0,0.0
282,09-01-24,1.09313,1.09730,1.09230,1.09727,53.223704,NaN,exit short,-1,-1.0,-2.0
284,11-01-24,1.09736,1.09870,1.09360,1.09508,47.045467,NaN,exit long,-1,-1.0,-2.0
298,31-01-24,1.08176,1.08749,1.07800,1.08720,50.779340,NaN,exit short,-1,-1.0,-2.0


#### create new dataframe for collecting trade record and calculate pnl (profit and loss)

- ENTRY (buy or sell) -> sum = 0 or 1

- EXIT (exit long or exit short) -> sum = 0

- NO ACTION -> sum = -1 or -2 or 2

In [9]:
def trade (df,lot=0.1):
    trade_records = pd.DataFrame(columns=['entry_time', 'exit_time', 'signal', 'entry_price', 'exit_price','profit','index'])

    for index in df.index:
        if df.loc[index, 'position_sum'] >= 0 and df.loc[index, 'position_sum'] < 2:
            if df.loc[index, 'action'] == 'buy':
                entry_time = df.time[index]
                entry_price = df.open[index]
                signal = 'buy'
                index = index    
                new_row = pd.DataFrame({'entry_time': [entry_time], 'entry_price': [entry_price], 'signal': signal, 'index': [index]})
                trade_records = pd.concat([trade_records, new_row], ignore_index=True)
                past_index = index
                
            elif df.loc[index, 'action'] == 'sell':
                entry_time = df.time[index]
                entry_price = df.open[index]
                signal = 'sell'
                past_index = index        
                new_row = pd.DataFrame({'entry_time': [entry_time], 'entry_price': [entry_price], 'signal': signal, 'index': [index]})
                trade_records = pd.concat([trade_records, new_row], ignore_index=True)
                
            elif df.loc[index, 'exit'] == 'exit long':
                exit_time = df.time[index]
                exit_price = df.close[index]
                index = past_index      
                new_row = pd.DataFrame({'exit_time': [exit_time], 'exit_price': [exit_price], 'index': [index]})
                trade_records = pd.concat([trade_records, new_row], ignore_index=True)
                
            elif df.loc[index, 'exit'] == 'exit short':
                exit_time = df.time[index]
                exit_price = df.close[index]
                index = past_index
                new_row = pd.DataFrame({'exit_time': [exit_time], 'exit_price': [exit_price], 'index': [index]})
                trade_records = pd.concat([trade_records, new_row], ignore_index=True)
        
    
        agg_functions = {'entry_time': 'first', 'exit_time': 'last', 'signal': 'first', 'entry_price': 'first', 'exit_price': 'last'}
        
        trade_df = trade_records.groupby(trade_records['index']).agg(agg_functions).reset_index(drop=True)
        
        pnl_buy = (trade_df['exit_price'] - trade_df['entry_price'])*lot
        pnl_sell = (trade_df['entry_price'] - trade_df['exit_price'])*lot
        
        trade_df.loc[trade_df['signal'] == 'buy', 'profit'] = pnl_buy
        trade_df.loc[trade_df['signal'] == 'sell', 'profit'] = pnl_sell
                        
    return trade_df

trade dataframe with pnl calculation

In [10]:
trade(action_df)

C:\Users\Admin\AppData\Local\Temp\ipykernel_20124\65860359.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_records = pd.concat([trade_records, new_row], ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_20124\65860359.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_records = pd.concat([trade_records, new_row], ignore_index=True)


,entry_time,exit_time,signal,entry_price,exit_price,profit
0,29-12-22,02-01-23,buy,1.06610,1.05490,-0.001120
1,11-01-23,02-02-23,buy,1.07579,1.07940,0.000361
2,21-02-23,28-02-23,sell,1.06464,1.06696,-0.000232
3,21-03-23,30-04-23,buy,1.07694,1.09770,0.002076
4,11-05-23,07-06-23,sell,1.09158,1.07828,0.001330
5,14-06-23,28-06-23,buy,1.08316,1.08644,0.000328
6,11-07-23,23-07-23,buy,1.10087,1.10648,0.000561
7,01-08-23,28-08-23,sell,1.09848,1.08788,0.001060
8,04-09-23,05-10-23,sell,1.07966,1.05872,0.002094
9,02-11-23,29-11-23,buy,1.06224,1.08884,0.002660


#### total sum of pnl (lot size comparison)

sum profit for lot size 0.1, 1, 10, 100

In [16]:
profit0 = trade(action_df,0.1)['profit'].sum()
profit1 = trade(action_df,1)['profit'].sum()
profit10 = trade(action_df,10)['profit'].sum()
profit100 = trade(action_df,100)['profit'].sum()

print('lot size: 0.1 -> profit: ' + str(profit0)) 
print('lot size: 1 -> profit: ' + str(profit1))
print('lot size: 10 -> profit: ' + str(profit10))
print('lot size: 100 -> profit: ' + str(profit100))


C:\Users\Admin\AppData\Local\Temp\ipykernel_20124\65860359.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_records = pd.concat([trade_records, new_row], ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_20124\65860359.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_records = pd.concat([trade_records, new_row], ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_20124\65860359.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or

lot size: 0.1 -> profit: 0.009004999999999997
lot size: 1 -> profit: 0.09004999999999996
lot size: 10 -> profit: 0.9004999999999996
lot size: 100 -> profit: 9.004999999999995


lot = 0.01

In [11]:
trade(action_df,0.01)['profit'].sum()

C:\Users\Admin\AppData\Local\Temp\ipykernel_20124\65860359.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_records = pd.concat([trade_records, new_row], ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_20124\65860359.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_records = pd.concat([trade_records, new_row], ignore_index=True)


0.0009004999999999997

lot = 1

In [ ]:
trade(action_df,1)['profit'].sum()

C:\Users\Admin\AppData\Local\Temp\ipykernel_14816\65860359.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_records = pd.concat([trade_records, new_row], ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14816\65860359.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_records = pd.concat([trade_records, new_row], ignore_index=True)


0.09004999999999996

lot = 10

In [ ]:
trade(action_df,10)['profit'].sum()

C:\Users\Admin\AppData\Local\Temp\ipykernel_14816\65860359.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_records = pd.concat([trade_records, new_row], ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14816\65860359.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_records = pd.concat([trade_records, new_row], ignore_index=True)


0.9004999999999996

lot = 100

In [ ]:
trade(action_df,100)['profit'].sum()

C:\Users\Admin\AppData\Local\Temp\ipykernel_14816\65860359.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_records = pd.concat([trade_records, new_row], ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14816\65860359.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_records = pd.concat([trade_records, new_row], ignore_index=True)


9.004999999999995